# Quantum Volume

In [ ]:
from qiskit_experiments.library import QuantumVolume
from qiskit_experiments.framework import BatchExperiment
from qiskit import transpile, execute
from qiskit.circuit.library import QuantumVolume as QuantumVolumeCircuit
from qiskit.quantum_info import Statevector

from huayi_providers.fake_huayi37 import FakeHuayi37, FakeHuayi37V2
from huayi_providers.fake_QuantinuumH2 import FakeQuantinuumh2, FakeQuantinuumh2V2
from qiskit_ionq import IonQProvider
from qiskit.providers.fake_provider import *
from qiskit_aer.noise.noise_model import NoiseModel

from qiskit import Aer
from qiskit_aer import AerSimulator

import pandas as pd

## Use qiskit-experiments built-in QV measurement

**Error message**

Don't use V1 backend (including FakeHuayi12, FakeMontreal, ...)

something wrong with:

``transpiled_circuits = experiment._transpiled_circuits()``


There is also an unknown problem when n_qubits or trials is too big when use V2 backend

some discussions:

https://github.com/Qiskit-Extensions/qiskit-experiments/issues/846


In [ ]:
def collect_data(batch_data):
    results_table = pd.DataFrame()
    for i in range(batch_data.num_experiments):
        qv_result = batch_data.child_data(i).analysis_results("quantum_volume")
        data = {'QV': qv_result.value} | {'quality': qv_result.quality} | qv_result.extra
        results_table = pd.concat([results_table, pd.DataFrame(data, index=[i])], ignore_index=True)
    return results_table

In [ ]:
simulator_huayi = FakeHuayi37V2()
simulator_montreal = FakeMontrealV2()

# Each QuantumVolume() is an Experimet class
# The length of tuple() is the circuit depth
exps = [QuantumVolume(tuple(range(i)), trials=100) for i in range(3,7)]

# Batch the experiments (with different depth) into one object
batch_exp = BatchExperiment(exps)
batch_exp.set_transpile_options(optimization_level=3)

# Run with the specified backend
data_huayi = batch_exp.run(simulator_huayi).block_for_results()
data_montreal = batch_exp.run(simulator_montreal).block_for_results()

# Collect the results in the 'results_table'
results_huayi = collect_data(data_huayi)
results_montreal = collect_data(data_montreal)

# results_table = pd.DataFrame()
# for i in range(batch_exp.num_experiments):
#     qv_result = data_huayi.child_data(i).analysis_results("quantum_volume")
#     data = {'QV': qv_result.value} | {'quality': qv_result.quality} | qv_result.extra
#     results_table = pd.concat([results_table, pd.DataFrame(data, index=[i])], ignore_index=True)

display(results_huayi)
display(data_huayi.child_data(batch_exp.num_experiments-1).figure(0))

display(results_montreal)
display(data_montreal.child_data(batch_exp.num_experiments-1).figure(0))

In [ ]:
# Single run

# Each QuantumVolume() is an Experimet class
# The length of tuple() is the circuit depth
exps = QuantumVolume(tuple(range(4)), trials=160) 

# Batch the experiments (with different depth) into one object
exps.set_transpile_options(optimization_level=3)

backend = AerSimulator.from_backend(FakeMontrealV2())
# Run with the specified backend
expdata = exps.run(backend).block_for_results()

expdata.figure(0)

## From qiskit textbook

https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware/measuring-quantum-volume.ipynb

Procedure:

1. Set ``ncircuits`` and ``nshots``, sweep ``depth``
2. Generate ``ncircuits`` random circuits
3. For each ``circuit``, simulate heavy-ouput, w/o and w/ noise. The noisy simulations are averaged over ``nshots`` times

In [ ]:
def get_heavy_outputs(counts):
    """Extract heavy outputs from counts dict.
    Args:
        counts (dict): Output of `.get_counts()`
    Returns:
        list: All states with measurement probability greater
              than the mean.
    """
    # sort the keys of `counts` by value of counts.get(key)
    sorted_counts = sorted(counts.keys(), key=counts.get)
    # discard results with probability < median
    heavy_outputs = sorted_counts[len(sorted_counts)//2:]
    return heavy_outputs

def check_threshold(nheavies, ncircuits, nshots):
    """Evaluate adjusted threshold inequality for quantum volume.
    Args:
        nheavies (int): Total number of heavy outputs measured from device
        ncircuits (int): Number of different square circuits run on device
        nshots (int): Number of shots per circuit
    Returns:
        Bool:
            True if heavy output probability is > 2/3 with 97% certainty,
            otherwise False
    """
    from numpy import sqrt
    numerator = nheavies - 2*sqrt(nheavies*(nshots-(nheavies/ncircuits)))
    return bool(numerator/(ncircuits*nshots) > 2/3)


def test_qv(device, nqubits, ncircuits, nshots):
    """Try to achieve 2**nqubits quantum volume on device.
    Args:
        device (qiskit.providers.Backend): Device to test.
        nqubits (int): Number of qubits to use for test.
        ncircuits (int): Number of different circuits to run on the device.
        nshots (int): Number of shots per circuit.
    Returns:
        Bool
            True if device passes test, otherwise False.
    """
    from numpy import sqrt

    def get_ideal_probabilities(circuit):
        """Simulates circuit behaviour on a device with no errors."""
        state_vector = Statevector.from_instruction(
                circuit.remove_final_measurements(inplace=False)
            )
        return state_vector.probabilities_dict()

    def get_real_counts(circuit, backend, shots):
        """Runs circuit on device and returns counts dict."""
        t_circuit = transpile(circuit, backend, optimization_level=3)
        job = backend.run(t_circuit,
                          shots=shots,
                          memory=True)
        return job.result().get_counts(), t_circuit

    # generate set of random circuits
    qv_circuits = [
        QuantumVolumeCircuit(nqubits) for c in range(ncircuits)
    ]

    nheavies = [0]*ncircuits  # number of measured heavy outputs
    cum_HOP = [0]*ncircuits  # cumulant heavy-output percentage
    cum_2sigma = [0]*ncircuits  # cumulant 2-sigma deviation
    transpiled_circuits = []
    for i, circuit in enumerate(qv_circuits):
        # simulate circuit
        ideal_heavy_outputs = get_heavy_outputs(
            get_ideal_probabilities(circuit)
        )
        # run circuit on device
        circuit.measure_all()
        real_counts, t_circuit = get_real_counts(circuit, device, nshots)
        transpiled_circuits.append(t_circuit)
        # record whether device result is in the heavy outputs
        for output, count in real_counts.items():
            if output in ideal_heavy_outputs:
                nheavies[i] += count
        cum_HOP[i] = sum(nheavies[0:i+1]) / nshots / (i+1)
        cum_2sigma[i] = 2 * sqrt( cum_HOP[i] * ( 1 - cum_HOP[i] ) / (i+1) )

    # do statistical check to see if device passes test
    is_pass = check_threshold(sum(nheavies), ncircuits, nshots)
    # calculate percentage of measurements that are heavy outputs
    percent_heavy_outputs = sum(nheavies)*100/(ncircuits*nshots)

    results = {
        "backend":device.name(),
        "n_qubits":nqubits,
        "QV": 2**nqubits, 
        "HOP": percent_heavy_outputs, 
        "success": is_pass, 
        "n_circuits": ncircuits,
        "n_shots": nshots,
        "cum_HOP":cum_HOP,
        "cum_2sigma":cum_2sigma}

    print(f"Quantum Volume: {2**nqubits}\n"
          f"Percentage Heavy Outputs: {percent_heavy_outputs:.1f}%\n"
          f"Passed?: {is_pass}\n")
    return results, qv_circuits, transpiled_circuits


In [ ]:
ncircuits = 100
nshots = 10

result, qv_circs, tr_circs = test_qv(FakeQuantinuumh2(), 6, ncircuits=ncircuits, nshots=nshots)

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ax.scatter(range(ncircuits), result["cum_HOP"], s=6, c='r')
ax.fill_between(range(ncircuits), 
                np.array(result["cum_HOP"]) - np.array(result["cum_2sigma"]), 
                np.array(result["cum_HOP"]) + np.array(result["cum_2sigma"]), color='b', alpha=0.4)
ax.hlines(2/3, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='k')
ax.set_ylim([0.5,1])


In [ ]:
import os
import json
import pandas as pd

if not os.path.isdir("./QV_test"):
    os.mkdir("./QV_test")

ncircuits = 200
nshots = 20

results_huayi = pd.DataFrame()
results_quantinuum = pd.DataFrame()

for depth in range(1,11):

    result, qv_circs, tr_circs = test_qv(FakeQuantinuumh2(), depth, ncircuits=ncircuits, nshots=nshots)
    results_quantinuum = pd.concat([result, pd.DataFrame(result, columns=["backend", "n_qubits", "QV", "HOP", "success", "n_circuits", "n_shots"], index=[0])], ignore_index=True)
    qv_export = result | {"QV_circuits": [c.qasm() for c in qv_circs]} | {"transpiled_circuits": [c.qasm() for c in tr_circs]}
    
    with open("QV_test/QV_FakeQuantinuumh2_{}.json".format(depth), 'w') as f:
        json.dump(qv_export, f)



for depth in range(1,11):

    result, qv_circs, tr_circs = test_qv(FakeHuayi37(), depth, ncircuits=ncircuits, nshots=nshots)
    results_huayi = pd.concat([result, pd.DataFrame(result, columns=["backend", "n_qubits", "QV", "HOP", "success", "n_circuits", "n_shots"], index=[0])], ignore_index=True)
    qv_export = result | {"QV_circuits": [c.qasm() for c in qv_circs]} | {"transpiled_circuits": [c.qasm() for c in tr_circs]}
    
    with open("QV_test/QV_FakeHuayi37_{}.json".format(depth), 'w') as f:
        json.dump(qv_export, f)